<a href="https://colab.research.google.com/github/sku1978/sk-share-repo/blob/main/Spark/SparkDataFrame/SparkDataFrameNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq  > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz > /dev/null 
!pip install -q findspark

In [2]:
!mkdir /content/conf /content/lib
!wget -O /content/conf/log4j.properties https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/log4j.properties > /dev/null 2>&1
!mv /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf.bk  > /dev/null 2>&1
!wget -O /content/spark-3.1.1-bin-hadoop3.2/conf/spark-defaults.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark-defaults.conf  > /dev/null 2>&1
!wget -O /content/conf/spark.conf https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/conf/spark.conf > /dev/null 2>&1

!wget -O /content/lib/logger.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/logger.py  > /dev/null 2>&1
!wget -O /content/lib/utils.py https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/lib/utils.py  > /dev/null 2>&1

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop3.2'

**Spark UI section**
<br>To use Spark UI, uncomment below sections and use the public link

In [4]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip
#get_ipython().system_raw('./ngrok http 4050 &')

In [5]:
#!curl -s http://localhost:4040/api/tunnels

**Main Section**

In [6]:
from pyspark.sql import *
from pyspark import SparkConf, SparkFiles
from lib.logger import Log4J
from lib.utils import get_spark_app_config

conf=get_spark_app_config()

spark = SparkSession.builder\
        .config(conf=conf)\
        .enableHiveSupport() \
        .getOrCreate()

logger = Log4J(spark)

**Basic CSV Read**

In [7]:
def load_csv_file(spark, url):
   spark.sparkContext.addFile(url)

   survey_df=spark.read \
   .format("csv") \
   .option("header", "true") \
   .option("inferSchema", "true") \
   .option("mode", "FAILFAST") \
   .load('file://'+SparkFiles.get("sample.csv"))

   return survey_df

def count_by_country(survey_df):
  count_df= survey_df.select("Age", "Gender", "Country", "State") \
                     .where("Age <= 40") \
                     .groupBy("Country") \
                     .count()
  return count_df

In [8]:
logger.info("Start CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/sample.csv'

survey_df=load_csv_file(spark, url)
partitioned_survey_df=survey_df.repartition(2)

count_df=count_by_country(partitioned_survey_df)

logger.info(count_df.collect())

logger.info("End CSV Load")

**Data Types**<br>
(https://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.types)

**CSV Read (CSV)**

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType, StringType

logger.info("Start Flight Time CSV Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.csv'

spark.sparkContext.addFile(url)

flightSchemaStruct = StructType([StructField("FL_DATE", DateType(), True),
                                 StructField("OP_CARRIER", StringType(), True),
                                 StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
                                 StructField("ORIGIN", StringType(), True),
                                 StructField("ORIGIN_CITY_NAME", StringType(), True),
                                 StructField("DEST", StringType(), True),
                                 StructField("DEST_CITY_NAME", StringType(), True),
                                 StructField("CRS_DEP_TIME", IntegerType(), True),
                                 StructField("DEP_TIME", IntegerType(), True),
                                 StructField("WHEELS_ON", IntegerType(), True),
                                 StructField("TAXI_IN", IntegerType(), True),
                                 StructField("CRS_ARR_TIME", IntegerType(), True),
                                 StructField("ARR_TIME", IntegerType(), True),
                                 StructField("CANCELLED", IntegerType(), True),
                                 StructField("DISTANCE", StringType(), True),
                                ])

flight_time_df=spark.read \
                    .format("csv") \
                    .option("header", "true") \
                    .option("mode", "FAILFAST") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaStruct) \
                    .load('file://'+SparkFiles.get("flight-time.csv"))

flight_time_df.show()

logger.info("End CSV Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**JSON Read**

In [10]:
logger.info("Start Flight Time JSON Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.json'

spark.sparkContext.addFile(url)

flightSchemaDDL = """FL_DATE DATE, 
                     OP_CARRIER STRING, 
                     OP_CARRIER_FL_NUM INT,
                     ORIGIN STRING,
                     ORIGIN_CITY_NAME STRING,
                     DEST STRING,
                     DEST_CITY_NAME STRING,
                     CRS_DEP_TIME INT,
                     DEP_TIME INT,
                     WHEELS_ON INT,
                     TAXI_IN INT,
                     CRS_ARR_TIME INT,
                     ARR_TIME INT,
                     CANCELLED INT,
                     DISTANCE STRING"""

flight_time_df=spark.read \
                    .format("json") \
                    .option("dateFormat", "M/d/y") \
                    .schema(flightSchemaDDL) \
                    .load('file://'+SparkFiles.get("flight-time.json"))

flight_time_df.show()

logger.info("End JSON Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**Parquet Read**

In [11]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

flight_time_df.show()

logger.info("End Parquet Load")

+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME|DEST|DEST_CITY_NAME|CRS_DEP_TIME|DEP_TIME|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|CANCELLED|DISTANCE|
+----------+----------+-----------------+------+----------------+----+--------------+------------+--------+---------+-------+------------+--------+---------+--------+
|2000-01-01|        DL|             1451|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1115|    1113|     1343|      5|        1400|    1348|        0|     946|
|2000-01-01|        DL|             1479|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1315|    1311|     1536|      7|        1559|    1543|        0|     946|
|2000-01-01|        DL|             1857|   BOS|      Boston, MA| ATL|   Atlanta, GA|        1415|    1414|     1642|      9|        1721|    1651|        0|     946

**Write Avro file**<br>
Added <br>
spark.jars.packages                org.apache.spark:spark-avro_2.12:3.0.0
<br>to<br>
conf/spark-defaults.conf

In [12]:
from pyspark.sql.functions  import spark_partition_id

logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("End Parquet Load")
logger.info("Number of partitions: " + str(flight_time_df.rdd.getNumPartitions()))
#Even though there are two partitions, since the records are less, only one partition is used. Can repartition to get more partitions in o/p
logger.info("Records per partition: " + str(flight_time_df.groupBy(spark_partition_id()).count().collect()))

logger.info("Start Avro write")

flight_time_df.write \
              .format("avro") \
              .mode("overwrite") \
              .option("path", "dataSink/avro/") \
              .save()

logger.info("End Avro write")

In [13]:
!ls -l dataSink/avro/

total 13088
-rw-r--r-- 1 root root 13400431 Apr  8 22:28 part-00000-5ba35193-7904-402a-af7e-2ca027c43df0-c000.avro
-rw-r--r-- 1 root root        0 Apr  8 22:28 _SUCCESS


**Partioned Write**

In [14]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("Start Partitioned JSON write")

#To check file split at 10K records, look for this example
#!wc -l dataSink/json/OP_CARRIER\=DL/ORIGIN\=ATL/*
flight_time_df.write \
              .format("json") \
              .mode("overwrite") \
              .option("path", "dataSink/json/") \
              .partitionBy("OP_CARRIER", "ORIGIN") \
              .option("maxRecordsPerFile", 10000) \
              .save()

logger.info("End Partitioned JSON write")

In [15]:
!ls -l dataSink/json/

total 40
drwxr-xr-x  99 root root 4096 Apr  8 22:28 'OP_CARRIER=AA'
drwxr-xr-x  36 root root 4096 Apr  8 22:28 'OP_CARRIER=AS'
drwxr-xr-x  84 root root 4096 Apr  8 22:28 'OP_CARRIER=CO'
drwxr-xr-x 118 root root 4096 Apr  8 22:28 'OP_CARRIER=DL'
drwxr-xr-x  53 root root 4096 Apr  8 22:28 'OP_CARRIER=HP'
drwxr-xr-x 119 root root 4096 Apr  8 22:28 'OP_CARRIER=NW'
drwxr-xr-x  81 root root 4096 Apr  8 22:28 'OP_CARRIER=TW'
drwxr-xr-x 106 root root 4096 Apr  8 22:28 'OP_CARRIER=UA'
drwxr-xr-x  90 root root 4096 Apr  8 22:28 'OP_CARRIER=US'
drwxr-xr-x  58 root root 4096 Apr  8 22:28 'OP_CARRIER=WN'
-rw-r--r--   1 root root    0 Apr  8 22:28  _SUCCESS


**Managed tables**<br>
Config has been setup to write tables into *warehouse_location* folder

In [22]:
logger.info("Start Flight Time Parquet Load")

url='https://raw.githubusercontent.com/sku1978/sk-share-repo/main/Spark/SparkDataFrame/data/flight-time.parquet'

spark.sparkContext.addFile(url)

flight_time_df=spark.read \
                    .format("parquet") \
                    .option("inferSchema", "true") \
                    .load('file://'+SparkFiles.get("flight-time.parquet"))

logger.info("Create database")
spark.sql("CREATE DATABASE IF NOT EXISTS AIRLINES_DB")
spark.catalog.setCurrentDatabase("AIRLINES_DB")

logger.info("Write table")
flight_time_df.write \
              .mode("overwrite") \
              .saveAsTable("flight_data_tbl")

Py4JError: ignored

In [21]:
!ls -l metastore_db/

total 28
-rw-r--r-- 1 root root    4 Apr  8 22:31 dbex.lck
-rw-r--r-- 1 root root   38 Apr  8 22:36 db.lck
drwxr-xr-x 2 root root 4096 Apr  8 22:31 log
-rw-r--r-- 1 root root  608 Apr  8 22:31 README_DO_NOT_TOUCH_FILES.txt
drwxr-xr-x 2 root root 4096 Apr  8 22:31 seg0
-rw-r--r-- 1 root root  900 Apr  8 22:31 service.properties
drwxr-xr-x 2 root root 4096 Apr  8 22:36 tmp


**View Log**

In [16]:
!cat app-logs/sparklog.log

21/04/08 22:27:46 INFO Hello World: Start CSV Load
21/04/08 22:27:55 INFO Hello World: [[United Kingdom, 1], [Canada, 2], [United States, 4]]
21/04/08 22:27:55 INFO Hello World: End CSV Load
21/04/08 22:27:55 INFO Hello World: Start Flight Time CSV Load
21/04/08 22:27:58 INFO Hello World: End CSV Load
21/04/08 22:27:59 INFO Hello World: Start Flight Time JSON Load
21/04/08 22:28:04 INFO Hello World: End JSON Load
21/04/08 22:28:04 INFO Hello World: Start Flight Time Parquet Load
21/04/08 22:28:06 INFO Hello World: End Parquet Load
21/04/08 22:28:06 INFO Hello World: Start Flight Time Parquet Load
21/04/08 22:28:06 INFO Hello World: End Parquet Load
21/04/08 22:28:07 INFO Hello World: Number of partitions: 2
21/04/08 22:28:07 INFO Hello World: Records per partition: [Row(SPARK_PARTITION_ID()=0, count=470477)]
21/04/08 22:28:07 INFO Hello World: Start Avro write
21/04/08 22:28:12 INFO Hello World: End Avro write
21/04/08 22:28:13 INFO Hello World: Start Flight Time Parquet Load
21/04/08 